### Install deps

In [1]:
%pip install uv dotenv
!uv pip install datasets transformers sentencepiece peft accelerate bitsandbytes trl xformers nltk rouge-score evaluate
!uv pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

A saída de streaming foi truncada nas últimas 5000 linhas.
bitsandbytes ------------------------------ 4.48 MiB/72.54 MiB
nvidia-cusolver-cu12 ------------------------------ 4.67 MiB/122.01 MiB
nvidia-cusparse-cu12 ------------------------------ 5.01 MiB/197.84 MiB
nvidia-cufft-cu12 ------------------------------ 4.76 MiB/201.66 MiB
nvidia-cublas-cu12 ------------------------------ 4.56 MiB/346.60 MiB
   Building rouge-score==0.1.2
⠧ Preparing packages... (7/20)
datasets   ------------------------------ 334.81 KiB/475.97 KiB
nvidia-cuda-cupti-cu12 ------------------------------ 4.97 MiB/13.17 MiB
nvidia-nvjitlink-cu12 ------------------------------ 4.95 MiB/20.09 MiB
nvidia-cuda-nvrtc-cu12 ------------------------------ 5.23 MiB/23.50 MiB
xformers   ------------------------------ 4.44 MiB/41.35 MiB
nvidia-curand-cu12 ------------------------------ 4.90 MiB/53.70 MiB
bitsandbytes ------------------------------ 4.48 MiB/72.54 MiB
nvidia-cusolver-cu12 ------------------------------ 4.67 M

In [2]:
import dotenv
import os
from google.colab import userdata
from huggingface_hub import login
from google.colab import drive
from datasets import load_dataset

# mount google drive
drive.mount('/content/drive')

# envs
os.environ["WANDB_DISABLED"] = "true"
os.environ['UNSLOTH_RETURN_LOGITS'] = '1'
DRIVE = "/content/drive/MyDrive/amz-files"
model_checkpoint = "mistralai/Mistral-7B-v0.1"
REPO_ID="carlosrian/Mistral-7B-v0.1-finetuned-amazon-reviews-2.8k"
OUTPUT_DIR = f"{DRIVE}/{model_checkpoint}-finetuned-amazon-reviews-2.8k"

# login huggingface_hub
login(token=userdata.get('HF_TOKEN'))

# Read Data Using Dataset lib
raw_dataset = load_dataset('json', data_files={'train': f'{DRIVE}/clean_dataset.json'}, split='train')
raw_dataset

Mounted at /content/drive


Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['title', 'description'],
    num_rows: 2749
})

### Tokenize Datasets

In [9]:
from unsloth import FastLanguageModel, is_bfloat16_supported
from transformers import TrainingArguments
from trl import SFTTrainer

max_seq_length = 1024
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_checkpoint,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

prompt = """
Below is the instruction that describes a product, Based on 'Input' you must provide the `Output`.

### Instruction:
You must use the 'Output' to answer the 'Input' question!

### Input:
{input}

### Output:
{output}
"""


EOS_TOKEN = tokenizer.eos_token

def preprocess_function(examples: dict):
    texts = [prompt.format(input=title, output=description) + EOS_TOKEN
            for title, description in zip(examples["title"], examples["description"])]
    return { "text" : texts}

num_proc = os.cpu_count()
print(f"Number of processors: {num_proc}")
tokenized_datasets = raw_dataset.map(preprocess_function, batched=True, num_proc=num_proc)

# remove columns unused
tokenized_datasets = tokenized_datasets.remove_columns(["title", "description"])
tokenized_datasets

### Train fine tuning model

In [4]:
max_steps = 1000  # Try at least 1000+ for meaningful training
learning_rate = 1e-4  # Lower LR slightly for better convergence

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=4,  # ✅ Adjust batch size as needed
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=2,  # ✅ Increased for stability
    num_train_epochs=3,
    max_steps=max_steps,  # ✅ Increased (20 was too low!)
    learning_rate=learning_rate,  # ✅ Lowered for better fine-tuning
    fp16=not is_bfloat16_supported(),
    bf16=is_bfloat16_supported(),
    logging_steps=100,
    optim="adamw_torch",
    weight_decay=0.01,
    lr_scheduler_type="linear",
    seed=3407
)

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = tokenized_datasets,
    args = training_args
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/2749 [00:00<?, ? examples/s]

In [6]:
# trainer the model
trainer.train()
# save the model and tokenizer
trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)
# publish the model and tokenizer
trainer.push_to_hub()
tokenizer.push_to_hub(repo_id=REPO_ID)

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 2,749 | Num Epochs = 5 | Total steps = 1,000
O^O/ \_/ \    Batch size per device = 6 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (6 x 2 x 1) = 12
 "-____-"     Trainable parameters = 41,943,040/7,000,000,000 (0.60% trained)


Step,Training Loss
100,1.486400
200,1.463700
300,1.304700
400,1.258700
500,1.109200
600,1.021200
700,0.975900
800,0.825600
900,0.801900
1000,0.711300


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

events.out.tfevents.1742828035.f3fbcf2046dd.445.0:   0%|          | 0.00/14.6k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.75k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


### Load remote and local model/tokenizer

In [ ]:
# load model and tokenizer from huggingface
from transformers import AutoTokenizer, AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("carlosrian/Mistral-7B-v0.1-finetuned-amazon-reviews-2.8k")
tokenizer = AutoTokenizer.from_pretrained("carlosrian/Mistral-7B-v0.1-finetuned-amazon-reviews-2.8k")

In [ ]:
# load model and tokenizer from local
from transformers import AutoTokenizer, AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(OUTPUT_DIR)
tokenizer = AutoTokenizer.from_pretrained(OUTPUT_DIR)

### Tests

In [34]:
def generate_response(title, model, tokenizer, max_length=200):
    input = prompt.format(input=title, output="")

    inputs = tokenizer(input, return_tensors="pt").to("cuda")

    output = model.generate(
        **inputs,
        max_length=max_length,
        temperature=0.8,  # Controls randomness (lower = more deterministic)
        top_k=50,         # Limits sampling to top 50 words
        top_p=0.95,       # Nucleus sampling (higher = more diverse)
        do_sample=True,   # Enables sampling instead of greedy decoding
    )

    return tokenizer.decode(output[0], skip_special_tokens=True)


In [22]:
title = "White Sierra Women's Sierra Point Convertible Pant (29-Inch Inseam)"
response = generate_response(title, model, tokenizer, max_length=500)
print("Generated Description:", response)


# From mountain trails to sunny river banks to scorching deserts,
# the Sierra Point Convertible Pant swiftly converts from pants to shorts when the temps change.
# The lightweight nylon Sierra Cloth woven fabric dries just as quickly.

Generated Description: 
Below is the instruction that describes a product, Based on 'Input' you must provide the `Output`.

### Instruction:
You must use the 'Output' to answer the 'Input' question!

### Input:
White Sierra Women's Sierra Point Convertible Pant (29-Inch Inseam)

### Output:

These durable nylon pants feature a zip-off pant section that instantly transforms to a 9.5" hemline. The other upper-leg is a 7.5" inseam short, both ideal for transitioning from activity to activity. UPF 30 protection and zip-closed security pocket.



In [30]:
title = "Zaggora Women's Hot Top"
response = generate_response(title, model, tokenizer, max_length=500)
print("Generated Description:", response)


# Zaggora Hot Tops - Small-Black - Hot Tops gets you hot in style - CELU-LITE Technology The Zaggora Hot Line of
# products has been extensively tested in both the US and UK for effectiveness.

Generated Description: 
Below is the instruction that describes a product, Based on 'Input' you must provide the `Output`.

### Instruction:
You must use the 'Output' to answer the 'Input' question!

### Input:
Zaggora Women's Hot Top

### Output:

Zaggora Women's Hot Tops offer you the same great heat-activated weight loss and cellulite reduction benefits as our other fashionable fitness apparel. Zaggora Hot Tops are made from our proprietary fabric which has been scientifically proven to increase circulation and blood flow to the muscles, which in turn helps you burn more calories during your workout. Zaggora Hot Tops can be worn during any type of exercise, from yoga to spin class, and even under your work clothes.



In [36]:
title = "Asics Men's Core Long Sleeve Shirt"
response = generate_response(title, model, tokenizer, max_length=500)
print("Generated Description:", response)

# The perfect base layer for a windy winter run, or the ideal sun-blocking top for summer.
# Whatever your choice, this top is up to the task, with revolutionary Hydrology construction that keeps the body comfortable and dry.

Generated Description: 
Below is the instruction that describes a product, Based on 'Input' you must provide the `Output`.

### Instruction:
You must use the 'Output' to answer the 'Input' question!

### Input:
Asics Men's Core Long Sleeve Shirt

### Output:

The perfect lightweight, long sleeved top for cool weather runs. Features moisture wicking and sun blocking fabric.



In [41]:
title = "Boxercraft Women's Fashion Flannel Pajama Pant"
response = generate_response(title, model, tokenizer, max_length=500)
print("Generated Description:", response)

# Be comfortable and cool in these awesome 100% cotton flannel pajama pants by Boxercraft.
# Features covered elastic waistband with imprintable taping and drawstring. Show off your school spirit or your fashion sense!

Generated Description: 
Below is the instruction that describes a product, Based on 'Input' you must provide the `Output`.

### Instruction:
You must use the 'Output' to answer the 'Input' question!

### Input:
Boxercraft Women's Fashion Flannel Pajama Pant

### Output:

These fantastic flannel pajama pants feature an elastic waistband with contrast stitching and poppets for a comfortable fit. The leg cuffs also have elastic to keep the pants up on the leg. Decorated with contrasting stripes down the outside leg, these pants are a great way to stay comfortable.

